In [1]:
# Script to calculate generality and  application data
# Generality: how diverse is the impact of a patent
# This is done by calculating the herfindal index of citing patents

In [2]:
# Feb 9th, 2020
# It's working and I'm keeping as v3
# Now I begin in v4 
# This can be used both for generality and originality
# also, I can use for multiple classes

# Feb 7th, 2020
# transforming this in a function

# Feb 7th, 2020
# While the previous version is already up, I want to improve the script
# I want to make calculations with only one script
# Also, I want to compare different class systems
# this version took 10 minutes - i am moving the old version away

# Jan 16th, 2020
# Due to performance problems in the HPC, this script was divided in two, the script following this is generality_2
# generality > 1 is not an error, but a consequence of adopting WIPO
# it seems that the original calculation had only one class per patent
# WIPO provides multiple classes - so when you divide by the total number of citations, 
#  you do not have the proportion of classes cited anymore
# to correct this issue, I can calculate Generality and Originality based only on the first WIPO class

# Jan 13th, 2020
# Script is running but there are two major issues
# - there should not exist generality > 1 , so there is an error in calculation
# - too many NANs (about 400k), but I will tackle this issue in 'too_many_nans.ipynb'

# to tackle the first problem, I'll begin by creating a subset of the database
# to do that, I'll use USPTO classification system

In [3]:
# Trying to save memory is leading to a small nightmare 
# I am postponing the use of dask modules

In [4]:
import pandas as pd
import numpy as np
import re

In [5]:
def preprocessing(class_df, citation_df):
    #there is a bug in pandas.read_csv
    #it fails to classify as object when told so in the options
    #so in this script, I guarantee the indexes are objects
    #then I match class to citation
    #inner matching in a citation-level returns citation level
    #i haven't tested but it could also be used to match class to patent

    #to guarantee same format for the merge
    class_df['id']=class_df['id'].astype(str)
    citation_df=citation_df.astype(str)

    #join on index is faster
    class_df.set_index('id', inplace=True)
    citation_df.set_index('patent_id', inplace=True)

    #citation level dataset
    #join is faster than merge
    df=citation_df.join(class_df, how='inner')  

    return df #citations with classifications

In [6]:
def generality(df):
    #the left dataframe is citation df, which is indexed by patent_id
    #when later on I group by citation_id is very possible that NaNs appear
    
    #considering the second level of class systems
    subclass=df.columns[2] #checar

    total_citation=df.groupby('citation_id').count().iloc[:,0]
    
    #citation-subclassclass level of observation
    df=df.groupby(['citation_id', subclass]).count()

    df_squared=np.square(df) #element-wise squaring
#     return total_citation, df_squared
#     df_squared=df_squared.sum(axis='columns') #sum all columns, per row
    df_squared=df_squared.reset_index().groupby('citation_id').sum()
#     df_squared.head()

    df2=pd.concat([df_squared, total_citation], axis=1)
    df2.columns=['df_squared', 'total_citation']

    df2['herfindal']=df2['df_squared']/df2['total_citation'] #its a measure of concentration

    df2['generality']=1-df2['herfindal'] # as defined in Hall et al, 2001

    return df2

In [7]:
citation= 'data/cleanuspatentcitation.csv'
classification = 'data/wipo.csv'
dst= 'data/generality_v3.csv'

class_df=pd.read_csv(classification, dtype=object)
citation_df=pd.read_csv(citation, sep=',', usecols=['patent_id', 'citation_id'])

In [8]:
df=preprocessing(class_df, citation_df)
df.head()

,citation_id,wipo_sector_id,wipo_field_id
3930271,2379430,4,33
3930271,2782422,4,33
3930272,2560109,4,33
3930272,2545289,4,33
3930272,1549144,4,33


In [9]:
df=generality(df)
df.head()

,df_squared,total_citation,herfindal,generality
citation_id,,,,
'799989,1,1,1.0,0.0
'935893,1,1,1.0,0.0
+0,1,1,1.0,0.0
+1850714,1,1,1.0,0.0
+3890950,1,1,1.0,0.0


In [ ]:
df.to_csv(dst)